In [ ]:
---
title: Solving 2D ODE by Matrix Exponentials
description: An application of eigenvalue decomposition
author: Daning H.
show-code: False
show-prompt: False
params:
    a11:
        input: numeric
        label: a_11
        value: 1.0
        min: -2
        max: 2
        step: 0.1
    a12:
        input: numeric
        label: a_12
        value: -1.0
        min: -2
        max: 2
        step: 0.1
    a21:
        input: numeric
        label: a_21
        value: -0.25
        min: -2
        max: 2
        step: 0.1
    a22:
        input: numeric
        label: a_22
        value: 1.0
        min: -2
        max: 2
        step: 0.1
    y01:
        input: numeric
        label: y_01
        value: 0.1
        min: -2
        max: 2
        step: 0.1
    y02:
        input: numeric
        label: y_02
        value: 0.0
        min: -2
        max: 2
        step: 0.1
    Dt:
        input: numeric
        label: Step size
        value: 0.2
        min: 0.1
        max: 1.0
        step: 0.1
---

We consider an initial value problem defined by a 2D ODE
$$
y'=Ay,\quad y(0) = y_0
$$
where $y=[y_1,y_2]^T$ and
$$
A=\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{bmatrix}
$$
Suppose one has the eigenvalue decomposition
$
A=X\Lambda X^{-1}
$
then one can compute the matrix exponential
$
\exp(A) = X\exp(\Lambda) X^{-1}
$
and the solution is
$
y(t) = \exp(A)y_0
$

Assume the eigenpairs of $A$ are $(x_1,\lambda_1)$ and $(x_2,\lambda_2)$, then the matrix exponential effectively deomposes the initial condition as
$$
y_0 = c_1 x_1 + c_2 x_2
$$
then the solution evolves according to the eigenvalues
$$
y(t) = c_1\exp(\lambda_1 t) x_1 + c_2\exp(\lambda_2 t) x_2
$$

*Think about how you would compute $c=[c_1, c_2]^T$*

In [18]:
a11 = 1.0
a12 = -1.0
a21 = -0.25
a22 = 1.0
y01 = 0.1
y02 = 0.0
Dt  = 0.2

In [24]:
import plotly.graph_objects as go
import numpy as np

N = 10
ts = np.arange(N)*Dt
A = np.array([
    [a11, a12],
    [a21, a22]])
y0 = np.array([y01, y02])
L, X = np.linalg.eig(A)
# EA = X.dot(np.diag(np.exp(L*Dt))).dot(np.linalg.inv(X))

c = np.linalg.solve(X, y0).reshape(-1,1) * np.exp(L.reshape(-1,1)*ts)
y1 = X[:,0].reshape(-1,1) * c[0]
y1 = np.hstack([np.array([[0],[0]]), y1])
y2 = X[:,1].reshape(-1,1) * c[1]
y2 = np.hstack([np.array([[0],[0]]), y2])
ys = y1 + y2

fig = go.Figure()

# Add traces, one for each slider step
for _i in range(1,N+1):
    _sl = _i==1
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="blue", width=2),
            name="Eigenvector 1", x=y1[0,:_i+1], y=y1[1,:_i+1], showlegend=_sl))
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="red", width=2),
            name="Eigenvector 2", x=y2[0,:_i+1], y=y2[1,:_i+1], showlegend=_sl))
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="black", width=2),
            name="Solution", x=ys[0,1:_i+1], y=ys[1,1:_i+1], showlegend=_sl))
    fig.add_trace(
        go.Scatter(
            visible=False, line=dict(color="gray", width=1, dash='dash'),
            name="Decomposition", showlegend=_sl,
            x=[y1[0,_i],ys[0,_i],y2[0,_i]], y=[y1[1,_i],ys[1,_i],y2[1,_i]]))
for _i in range(4):
    fig.data[_i].visible = True

# Create and add slider
steps = []
for i in range(1,N+1):
    step = dict(
        method="update",
        args=[{"visible": [False] * N*4}],
        label=f"{i}"
    )
    for j in range(4*i):
        step["args"][0]["visible"][j] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Step = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    xaxis_title="x",
    yaxis_title="y",
#     autosize=False,
#     width=800,
#     height=800
)

# fig.update_xaxes(range=[-2.0, 2.0])
# fig.update_yaxes(range=[-2.0, 2.0])

fig.show()